In [4]:
#Dependecies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import datetime
from sqlalchemy.orm import sessionmaker

#Scraping
import requests
from bs4 import BeautifulSoup
import re
import urllib.request as urllib2

# Webscrape Products and Prices

In [5]:
def get_href(page):
    pg=page
    link_ = urllib2.urlopen(pg)
    bs4_obj = BeautifulSoup(link_)
    suff= [a.get_attribute_list('href')[0] for a in bs4_obj.find_all('a',href=True) if a.get_attribute_list('href')[0][:3]=='/p/']
    return list(set(suff))

In [6]:
href_list = get_href('https://www.dreamwaretech.co.za/c/networking-equipment/routers/')

In [108]:
def Extract(page,links_href):
    #Collect Product name and Price from each href
    Product_name=[]
    Product_price=[]
    for link_ in links_href:
        prod_link = urllib2.urlopen(re.match("(.*?).za",page).group()+link_)
        prod_obj = BeautifulSoup(prod_link)
        Product_name.append(prod_obj.find_all('h3')[0].text)
        Product_price.append(prod_obj.find_all('h2')[0].text[1:])
    return pd.DataFrame(zip(Product_name,Product_price),columns=['Product_name','Product_price'])



In [8]:
Comp_price_df = Extract('https://www.dreamwaretech.co.za/c/networking-equipment/routers/',href_list)
Comp_price_df['Product_price']=Comp_price_df['Product_price'].astype('float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
Product_name     18 non-null object
Product_price    18 non-null float64
dtypes: float64(1), object(1)
memory usage: 416.0+ bytes


In [9]:
Comp_price_df

,Product_name,Product_price
0,Asus RT-AX92U (1-Pack) 6100 Tri-Band WiFi 6 (8...,5129.0
1,Tenda AC10 AC1200 Smart Dual-Band Gigabit WiFi...,739.0
2,Tenda 4G680 4G LTE 300Mbps with SIM Slot WiFi ...,1249.0
3,Asus RT-AX56U AX1800 Dual Band WiFi 6 (802.11a...,2499.0
4,Asus RT-AC57U AC1200 Dual Band WiFi Router,1339.0
5,Asus RT-AX88U AX6000 Dual Band WiFi 6 (802.11a...,7959.0
6,ASUS AC750 dual-Band Wi-Fi Router with four an...,619.0
7,RAVPOWER AC750 6700mAh Wireless Travel FileHub...,809.0
8,Tenda Nova 2 Pack MW5 AC1200 Dual Band Whole H...,1579.0
9,Eusso 2-Fxs+2-Fxo-Port Voip W/Vpn Router,3149.0


# Connect to MySql Database

In [230]:
#Read login credentials from external file.
with open('C:\\Users\\Marcus\\Desktop\\credentials.txt','r') as creds:
    all_lines = creds.readlines()

In [231]:
cnx = create_engine(all_lines[0] + '/networksales')    
df = pd.read_sql('SELECT * FROM product', cnx) #read the entire table

In [232]:
df

,product_id,product_name,product_tier,price
0,ASU01,Asus RT-AC57U AC1200 Dual Band Wifi Router,Networking,1330.00
1,DLI01,D-Link/LTE N300 Wifi Router,Networking,1670.00
2,RAV01,RAVPOWER AC750 6700mAh Wireless Travel,Networking,829.99
3,SMC01,SMC Portable 3G Router,Networking,594.99
4,TEN01,Tenda Nova 2 Pack MW5 AC1200 Dual Band Whole H...,Networking,1615.00
5,TEN02,Tenda 4G185 4G LTE Portable Mifi Router,Networking,1100.00
6,Ten03,Tenda F9 600Mbps Wifi Router and Repeater,Networking,450.00


# Compare Prices

In [115]:
def compare(my_prod,my_price,comp_prod,comp_price):
    prod=[]
    price=[]
    #Make function to compare values
    for name in df[my_prod]:
        if Comp_price_df[Comp_price_df[comp_prod].str.contains(pat=str(name[:-5]),case=False)][comp_price].values< \
        df[df[my_prod].str.contains(pat=str(name[:-5]),case=False)][my_price].values:
            prod.append(df[df[my_prod].str.contains(pat=str(name[:-5]),case=False)][my_prod].values)
            price.append(Comp_price_df[Comp_price_df[comp_prod].str.contains(pat=str(name[:-5]),case=False)][comp_price].values)
    return pd.DataFrame(zip(prod,price),columns=['Prod_alter','Price_alter'])


In [165]:
df_alter =compare('product_name','price','Product_name','Product_price')
df_alter['Adusted_Mark_up'] = df_alter['Price_alter'] - (df_alter['Price_alter']* 0.03 )
df_alter['Adusted_Mark_up'] = df_alter['Adusted_Mark_up'].astype('float')
df_alter['Prod_alter']=df_alter['Prod_alter'].apply(lambda x: x[0])

C:\Users\Marcus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  import sys


# Alter Prices on my "Price" table on MySQL

In [233]:
pd.read_sql_query('SELECT * FROM product',cnx)

,product_id,product_name,product_tier,price
0,ASU01,Asus RT-AC57U AC1200 Dual Band Wifi Router,Networking,1330.00
1,DLI01,D-Link/LTE N300 Wifi Router,Networking,1670.00
2,RAV01,RAVPOWER AC750 6700mAh Wireless Travel,Networking,829.99
3,SMC01,SMC Portable 3G Router,Networking,594.99
4,TEN01,Tenda Nova 2 Pack MW5 AC1200 Dual Band Whole H...,Networking,1615.00
5,TEN02,Tenda 4G185 4G LTE Portable Mifi Router,Networking,1100.00
6,Ten03,Tenda F9 600Mbps Wifi Router and Repeater,Networking,450.00


In [225]:
pd.read_sql_query("SELECT * from product WHERE product_name like 'MC Portable 3G Router'",cnx)

,product_id,product_name,product_tier,price


In [ ]:
sql_update ='UPDATE product SET price = {} WHERE product_name like *j[3:]'.format()

In [234]:
#Update MySQL DataBase
for i in df_alter['Prod_alter']:
    pr = df_alter[df_alter['Prod_alter'].str.contains(i[3:])]['Adusted_Mark_up'].values[0]
    sql_update =f"UPDATE product SET price = {pr} WHERE product_name like '{i}'"
    cnx.execute(sql_update)

In [236]:
#Read new table from MySQL
new_df = pd.read_sql('SELECT * FROM product', cnx)
new_df

,product_id,product_name,product_tier,price
0,ASU01,Asus RT-AC57U AC1200 Dual Band Wifi Router,Networking,1330.00
1,DLI01,D-Link/LTE N300 Wifi Router,Networking,1670.00
2,RAV01,RAVPOWER AC750 6700mAh Wireless Travel,Networking,784.73
3,SMC01,SMC Portable 3G Router,Networking,542.23
4,TEN01,Tenda Nova 2 Pack MW5 AC1200 Dual Band Whole H...,Networking,1531.63
5,TEN02,Tenda 4G185 4G LTE Portable Mifi Router,Networking,1100.00
6,Ten03,Tenda F9 600Mbps Wifi Router and Repeater,Networking,396.73


In [245]:
new_df_price = pd.concat([df['price'],new_df['price']],axis=1)
new_df_price.columns = ['Old_price','Updated_price']
new_df_price

,Old_price,Updated_price
0,1330.00,1330.00
1,1670.00,1670.00
2,829.99,784.73
3,594.99,542.23
4,1615.00,1531.63
5,1100.00,1100.00
6,450.00,396.73
